In [ ]:
#Step 1 - retrieve greek tweets real-time

import json
import tweepy
import pika
import sys
import time
import datetime
import re
import random
import os

#take Twitter keys
consumer_key = "<your_key>"
consumer_secret = "<your_key>"
access_key = "<your_key>"
access_secret = "<your_key>"

# set up the tweepy api feed
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#search by specific keywords (hashtags, words, letters)
keywords = ['coronavirus','covid19','#COVID19greece','α']


try:
    # Connect to the RabbitMQ server
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()

    # Create a queue called <your_queue_name>
    channel.queue_declare(queue='<your_queue_name>')

except Exception as err:
    print (err)


class CustomStreamListener(tweepy.StreamListener):
    count = 0
    # stop after 1000 tweets (optional - can be removed)
    total_tweets = 1000

    def __init__(self, api):
        self.api = api
        super(tweepy.StreamListener, self).__init__()

    def on_status(self, status):

        def clean_tweet(tweet):
            return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

        if ('RT @' not in status.text):

            if not status.truncated:
                text = status.text
            else:
                text = status.extended_tweet['full_text']
                
            print(text)    

            data = {
                'text': text,
                'user': status.user.name,
                'user_statuses_count': status.user.statuses_count,
                'user_followers': status.user.followers_count,
                'user_friends': status.user.friends_count,
                'listed_count':status.user.listed_count,
                'user_location': status.user.location,
                'user_verified': status.user.verified,
                'fav_count': status.favorite_count,
                'rt_count': status.retweet_count,
                'tweet_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            
            #keep messages only for 5sec
            channel.basic_publish(exchange='', routing_key='<your_queue_name>', properties=pika.BasicProperties(expiration='5000',), body=json.dumps(data))
            self.count += 1
            if self.count > self.total_tweets:
                return False

    def on_error(self, status_code):
        print ('An error has occurred! Status = %s' % status_code)

        return True 

    def on_timeout(self):
        print ('Snoozing...ZzZzZzzzz')

        return True 

    def on_disconnect(self, notice):
        print ('Disconnected: %s' % notice)
        return False  

# start the stream
sapi = tweepy.streaming.Stream(auth, CustomStreamListener(api), tweet_mode='extended')
# start filtering the stream
sapi.filter(languages=["el"], track=keywords, stall_warnings=True)